In [2]:
import os
import cv2
import numpy as np
from PIL import Image
import imageio

def video_to_frames(filename, resize_h=64, resize_w=64):
    """                                                                                                           
    Extract every frame from a video, and keep one in every 3 frames, and resize each frame.
    For original video: expect 90 frames per 3 second video, each frame 256-by-256.
    If the video has less than 90 frames, fill all remaining frames up to num_frames
    """
    # an example for filename: 'videos/skiing_train/flickr-0-0-0-1-1-5-9-7-2400011597_4.mp4'
    vidcap = cv2.VideoCapture(filename)
    frameCount = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frameWidth = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    #expect 30 frames to be extracted; if original video has less than
    #90 frames, fill the trailing ones with the last sampled frame
    reduced_frameCount = 30

    data = np.empty((reduced_frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))
    
    fc = 0
    success = True
    while (fc < min(frameCount, 90) and success):
        if (fc%3==0):
            success, data[fc//3, ...] = vidcap.read()
        else:
            vidcap.read()
        fc += 1
    if frameCount < 90-3:
        for i in range(frameCount//3+1, 30):
            data[i, ...] = data[frameCount//3, ...]
        
    #print('Finished reading video')
        
    # keep only one frame out of every 3 frames, and resize every frame
    data_resized = np.empty((reduced_frameCount, resize_h, resize_w, 3), np.float)
    for i in range(reduced_frameCount):
        img = Image.fromarray(data[i,...], 'RGB')
        img = img.resize((resize_h, resize_w), Image.ANTIALIAS)
        data_resized[i, ...] = np.asarray(img, dtype=np.float)
        # below line just for testing purpose, make sure to comment out after testing!
        ###img.save('extracted_images/skiing_test_1_frame%d.jpg' % i)
        
    data_first_last = np.empty((2, resize_h, resize_w, 3), np.float)
    data_first_last[0] = data_resized[0]
    data_first_last[1] = data_resized[-1]

        
    return data_resized, data_first_last

def generated_video_to_frames(filename, resize_h=64, resize_w=64):
    """                                                                                                           
    Extract every frame from a generated video.
    For original video: expect 30 frames per 3 second video, each frame 64-by-64.
    If the video has less than 90 frames, fill all remaining frames up to num_frames
    """
    vidcap = cv2.VideoCapture(filename)
    frameCount = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frameWidth = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    #expect 30 frames to be extracted
    reduced_frameCount = 30

    data = np.empty((reduced_frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))
    
    fc = 0
    success = True
    while success and fc < 30:
        success, data[fc] = vidcap.read()
        fc += 1
    return data


def video_distance(array_in, array_out):
    # convert to np arrays of shape [30,h,w,3] to compare L2 distance
    # in CVAE code: np.reduce_sum(tf.square(self.inputs - self.out), [1,2,3,4])
    distance = np.sqrt(np.sum(np.square(array_out - array_in)))
    return distance

def generate_resized_and_interpolated_video(file_in, file_out_reduced, file_out_interpolated):
    # make two 3s long videos at 10 fps by reducing original video size, and by
    # linear interpolation between 1st and last frame
    data, data_first_last = video_to_frames(file_in, resize_h=64, resize_w=64)
    h = data.shape[1]
    w = data.shape[2]
    
    data_interpolated = np.zeros([30,h,w,3])
    
    for i in range(30):
        data_interpolated[i] = data_first_last[0] + i/30 * (data_first_last[1] - data_first_last[0])
        
        
    data_interpolated = data_interpolated.astype(np.uint8)
    data_reduced = data.astype(np.uint8)
    
    #data_interpolated = data_interpolated.astype(np.uint8)
    imageio.mimwrite(file_out_interpolated, data_interpolated, fps=10)
    imageio.mimwrite(file_out_reduced, data_reduced, fps = 10)
    print("Interpolated video written out into: {}".format(file_out_interpolated))
    print("Reduced video written out into: {}".format(file_out_reduced))
    return data_interpolated, data_reduced
    
    
    
def L2difference(file_in, reduced_file_out, interpolated_file_out, generated_file_in):    
    # file_in = input original video file, (90,256,256,3)
    # reduced_file_out = output oirginal video in format (30,64,64,3)
    # interpolated_file_out = output interpolated video in format (30,64,64,3)
    # generated_file_in = input generated video in format (30,64,64,3)
    print("Original video is: {}".format(file_in))
    print("Generated video is: {}".format(generated_file_in))
    original, imgs = video_to_frames(file_in)
    generated = generated_video_to_frames(generated_file_in)
    interpolated, reduced = generate_resized_and_interpolated_video(file_in, reduced_file_out, interpolated_file_out)   
    print("Generated vs original video distance is")
    print(video_distance(original, generated))
    print("Interpolated vs original video distance is")
    print(video_distance(original, interpolated))
    return video_distance(original,generated), video_distance(original,interpolated)


In [3]:
# file_in_example = "videos/skiing_train/flickr-0-0-0-1-1-5-9-7-2400011597_4.mp4"
# file_out_example = "videos_interpolated/test_interpolated.mp4"

file_in2= "videos/erupting_val/getty-lava-erupting-from-earth-in-smoking-lava-field-hawaii-video-id1B02924_0004_7.mp4"
file_out2 = "videos_interpolated/test2.mp4"

generated_file_in = "CVAE_generated_0000.mp4"

# def main():
#     L2difference(file_in2, "aaaa_reduced.mp4", "aaaa_interpolated.mp4", "CVAE_generated_0000.mp4")
#     return 0



# main()

In [4]:
#### measure CVAE generated videos vs interpolated videos 


## CVAE test videos in order:

titlemix="""
0
getty-lava-erupting-from-earth-in-smoking-lava-field-hawaii-video-id1B02924_0004_7.mp4
1
giphy-qufwQbBoDhNYc_12.mp4
2
getty-close-up-fireworks-video-id628285946_1.mp4
3
flickr-5-3-9-9-6-7-3-0-5553996730_22.mp4
4
getty-view-of-heavy-explosion-in-tropical-island-video-id105186610_10.mp4
5
getty-clouds-pass-over-the-summit-of-the-erupting-oldoinyo-lengai-volcano-video-id3B2FBBC6_0597_82.mp4
6
meta-1395624_51.mp4
7
getty-close-up-of-firey-explosion-video-id307-291_1.mp4
8
getty-geysers-atacama-desert-video-id636281892_3.mp4
9
flickr-7-7-9-1-3-6-9-6-19077913696_14.mp4
10
getty-man-sits-on-ledge-overlooking-erupting-lava-lake-marum-volcano-ambrym-video-id150962447_2.mp4
11
getty-jaguar-crashes-through-a-concrete-barrier-video-id698871039_14.mp4
12
getty-large-and-dangerous-pyroclastic-flow-sweeps-down-the-side-of-sinabung-video-id478438408_4.mp4
13
getty-bardarbunga-volcano-eruption-and-lava-flow-video-id626439064_19.mp4
14
yt-Hj9wcztisDU_787.mp4
15
getty-geyser-erupts-and-splashes-video-id685738819_1.mp4
16
getty-volcanic-eruptions-on-mt-etna-sicily-italy-video-id157397360_10.mp4
17
getty-major-volcanic-eruption-and-ash-cloud-at-mount-sinabung-video-id498515125_7.mp4
18
giphy-1cdrsxfhNHfAQ_2.mp4
19
getty-film-can-the-birth-of-a-nation-clip-battle-field-lillian-gish-sitting-video-id509496583_9.mp4
20
getty-geysers-at-yellowstone-national-park-wyoming-usa-video-id83264515_16.mp4
21
flickr-7-0-0-3-7-1-1-8-21470037118_126.mp4
22
getty-eruption-on-mt-etna-13-october-98-sicily-italy-video-id157397349_2.mp4
23
getty-lava-from-a-volcanic-eruption-spews-into-the-night-sky-available-in-video-id3B2FBBA6_0005_67.mp4
24
getty-aerials-active-volcano-shinmoedake-video-id459964798_41.mp4
25
getty-eruzione-di-una-bocca-del-vulcano-stromboli-video-id847691400_13.mp4
26
giphy-ClG9uYb0hWzPq_0.mp4
27
getty-geyser-hole-filled-of-boiling-water-before-eruption-iceland-video-id456462830_6.mp4
28
getty-hot-water-spurting-from-geyser-yellowstone-national-park-wyoming-video-id141030434_2.mp4
29
getty-close-up-vigorously-erupting-lava-lake-in-marum-volcano-marum-volcano-video-id150962413_2.mp4
30
getty-flames-rise-from-a-volcanic-eruption-on-fernandina-island-available-video-id3B2FBBA3_0001_5.mp4
31
getty-aerial-track-towards-and-over-smoking-volcano-and-crater-video-id1B03393_0004_1.mp4
32
getty-big-ash-column-rises-from-shinmoedake-volcano-at-night-mount-in-of-video-id478689742_6.mp4
33
getty-el-volcan-calbuco-registro-dos-violentas-y-sorpresivas-erupciones-en-video-id471199520_34.mp4
34
getty-timelapsed-of-sunrise-at-mount-bromo-volcano-the-magnificent-view-of-video-id545326216_3.mp4
35
getty-the-voskok-1-becomes-the-first-manned-spaceflight-video-id450027634_46.mp4
36
flickr-1-3-3-7-8-6-3-3-18513378633_4.mp4
37
getty-yasur-volcano-video-id465612842_2.mp4
38
getty-algal-mat-grows-at-edge-of-hot-geyser-pool-yellowstone-usa-video-id564828225_9.mp4
39
getty-volcanic-ash-erupts-vigorously-from-the-crater-at-sakurajima-volcano-video-id695848030_12.mp4
40
getty-large-eruption-at-sakurajima-volcano-in-japan-sends-volcanic-ash-video-id464370844_11.mp4
41
getty-fighting-subsided-in-war-torn-gaza-monday-at-the-start-of-the-muslim-video-id453004362_19.mp4
42
getty--video-id104511618_18.mp4
43
getty-aerial-track-over-erupting-old-faithful-geyser-in-winter-yellowstone-video-id595896631_2.mp4
44
flickr-6-5-8-1-9-6-9-9-16365819699_53.mp4
45
getty-ash-explodes-from-crater-of-stromboli-volcano-close-up-shot-stromboli-video-id150962340_13.mp4
46
getty-medium-shot-girl-posing-with-blue-ribbon-next-to-model-of-erupting-video-id873-100_8.mp4
47
flickr-4-0-3-7-4-0-5-7-15940374057_128.mp4
48
getty-bubbling-flowing-redhot-lava-after-the-eruption-of-kilauea-volcano-video-id85200270_11.mp4
49
getty-bright-moon-shining-behind-eruption-plume-of-geyser-video-id685739069_19.mp4
50
an-erupting-volcano-sends-lava-spewing-into-the-sky-available-in-hd-video-id3B2FBBA6_0010.mp4
51
getty-sakurajima-volcano-erupts-violently-with-a-large-explosive-eruption-video-id464368814_13.mp4
52
getty-hot-spring-video-id185675476_5.mp4
53
getty-group-of-eoraptors-grazing-in-field-video-id1319-11_9.mp4
54
getty-boiling-water-ejects-from-erupting-castle-geyser-yellowstone-usa-video-id595897161_15.mp4
55
getty-close-up-large-waves-of-lava-erupting-from-volcano-marum-volcano-video-id150962405_3.mp4
56
getty-camera-pans-right-to-geyser-erupting-spraying-white-water-and-grey-video-id1B02172_0004_13.mp4
57
getty-grey-smoke-and-ash-cloud-rising-quickly-upwards-from-crater-zoom-out-video-id143541934_2.mp4
58
flickr-0-6-6-5-1-0-5-3-5906651053_10.mp4
59
giphy-Hssp6w4SfuOB2_10.mp4
60
flickr-9-7-6-6-1-2-1-7-9497661217_62.mp4
61
getty-smoke-and-flames-rise-from-ship-after-explosion-at-gadani-yard-on-1-video-id620102418_83.mp4
62
getty-clouds-scud-over-clepsydra-geyser-erupting-in-yellowstone-national-video-id3B2FBBED_0155_3.mp4
63
getty-eruption-of-volcano-bromo-in-indonesia-video-id461618460_4.mp4
64
getty-from-encyclopaedia-britannica-films-documentary-depicting-the-study-video-id85200310_4.mp4
65
getty-krakatau-video-id493322240_4.mp4
66
giphy-V05ReDqkS1VCg_4.mp4
67
flickr-0-5-7-3-1-3-2-1-21105731321_9.mp4
68
getty-mount-bromo-volcano-in-east-java-indonesia-video-id649447834_3.mp4
69
getty-mount-aso-video-id483913210_8.mp4
70
getty-geyser-in-iceland-video-id523120502_9.mp4
71
getty-sea-hell-in-beppu-japan-video-id473184667_1.mp4
72
getty-cliff-geyser-erupting-video-id685738891_4.mp4
73
5-1-1-5-7-8-7-1-6151157871_5.mp4
74
getty-close-up-shot-of-steam-pouring-from-crater-of-sakurajima-volcano-at-video-id150962351_11.mp4
75
flickr-1-2-7-2-1-9-4-2-2912721942_9.mp4
76
getty-mtbromo-time-lapse-video-id613371200_4.mp4
77
flickr-1-1-3-2-1-3-2-5-15911321325_26.mp4
78
getty-lava-flows-through-a-channel-available-in-hd-video-id3B2FBBA7_0002_4.mp4
79
getty-boardwalk-in-volcanic-wonderland-video-id472597513_7.mp4
80
getty-eruption-of-the-shinmoedake-crater-of-the-kirishima-volcano-japan-28-video-id150975548_14.mp4
81
getty-the-24story-monaco-tower-at-the-shuttered-riviera-hotel-casino-is-on-video-id591578470_2.mp4
82
flickr-4-6-7-8905106467_44.mp4
83
getty-firework-display-in-tokyo-video-id629717708_16.mp4
84
getty-close-up-of-steaming-lava-dome-at-merapi-volcano-central-java-29-video-id150962577_1.mp4
85
flickr-5-5-2-2-8-9-2-8-14255228928_19.mp4
86
getty-eruption-on-krakatau-video-id493455112_3.mp4
87
yt-SUNwIm3o-64_19.mp4
88
getty-cotopaxi-volcano-ecuador-erupting-water-vapour-in-november-2015-video-id506205424_4.mp4
89
getty-lava-surges-in-crater-at-dusk-marum-volcano-ambrym-island-vanuatu-video-id150962395_8.mp4
90
giphy-xT4uQbhogM1sS5njXi_0.mp4
91
getty-bright-explosion-with-fairydust-aftereffect-video-iddv389020_1.mp4
92
getty-aerial-view-of-molten-lava-flow-on-the-island-of-hawaii-video-id640164583_1.mp4
93
giphy-sUeU52JKc8one_2.mp4
94
yt-Y_9vd4HWlVA_404.mp4
95
getty-aerial-view-of-hot-lava-in-hawaii-volcanoes-national-park-video-id640164559_1.mp4
96
getty-exterior-night-shots-of-baltimore-police-advancing-with-riot-shields-video-id471929516_45.mp4
97
getty-aerials-active-volcano-aso-video-id459967068_1.mp4
98
getty-grey-smoke-and-ash-cloud-billow-from-crater-into-sky-zoom-in-to-mount-video-id143541944_89.mp4
99
getty-november-volcano-erupts-in-columbia-lib-columbia-extnight-nevada-del-video-id675358808_7.mp4
"""



In [5]:
titlelist = []
for line in titlemix.splitlines():
    if "mp4" in line:
        titlelist.append(line)
    
print(titlelist[0:3])

['getty-lava-erupting-from-earth-in-smoking-lava-field-hawaii-video-id1B02924_0004_7.mp4', 'giphy-qufwQbBoDhNYc_12.mp4', 'getty-close-up-fireworks-video-id628285946_1.mp4']


In [7]:
from shutil import copyfile


L2generated = np.zeros([100])
L2interpolated = np.zeros([100])

for i in range(len(titlelist)):
    L2generated[i], L2interpolated[i] = L2difference("videos/erupting_val/"+titlelist[i], 
                                               "videos_for_poster/eruption_test_{:02d}_reduced.mp4".format(i), 
                                               "videos_for_poster/eruption_test_{:02d}_interpolated.mp4".format(i), 
                                               "generated_TEST_videos/generated_CVAE_videos/CVAE_generated_00{:02d}.mp4".format(i))
    copyfile("generated_TEST_videos/generated_CVAE_videos/CVAE_generated_00{:02d}.mp4".format(i), "videos_for_poster/eruption_test_{:02d}_CVAE.mp4".format(i))
    
print(L2generated)
print(L2interpolated)




Original video is: videos/erupting_val/getty-lava-erupting-from-earth-in-smoking-lava-field-hawaii-video-id1B02924_0004_7.mp4
Generated video is: generated_TEST_videos/generated_CVAE_videos/CVAE_generated_0000.mp4
Interpolated video written out into: videos_for_poster/eruption_test_00_interpolated.mp4
Reduced video written out into: videos_for_poster/eruption_test_00_reduced.mp4
Generated vs original video distance is
10506.26060975074
Interpolated vs original video distance is
2203.3576650194586
Original video is: videos/erupting_val/giphy-qufwQbBoDhNYc_12.mp4
Generated video is: generated_TEST_videos/generated_CVAE_videos/CVAE_generated_0001.mp4
Interpolated video written out into: videos_for_poster/eruption_test_01_interpolated.mp4
Reduced video written out into: videos_for_poster/eruption_test_01_reduced.mp4
Generated vs original video distance is
11072.363388184114
Interpolated vs original video distance is
5441.941565287154
Original video is: videos/erupting_val/getty-close-up-fi

Interpolated video written out into: videos_for_poster/eruption_test_17_interpolated.mp4
Reduced video written out into: videos_for_poster/eruption_test_17_reduced.mp4
Generated vs original video distance is
9393.10922964276
Interpolated vs original video distance is
307.74827375632833
Original video is: videos/erupting_val/giphy-1cdrsxfhNHfAQ_2.mp4
Generated video is: generated_TEST_videos/generated_CVAE_videos/CVAE_generated_0018.mp4
Interpolated video written out into: videos_for_poster/eruption_test_18_interpolated.mp4
Reduced video written out into: videos_for_poster/eruption_test_18_reduced.mp4
Generated vs original video distance is
23270.0600987621
Interpolated vs original video distance is
768.2252273910302
Original video is: videos/erupting_val/getty-film-can-the-birth-of-a-nation-clip-battle-field-lillian-gish-sitting-video-id509496583_9.mp4
Generated video is: generated_TEST_videos/generated_CVAE_videos/CVAE_generated_0019.mp4
Interpolated video written out into: videos_for

Interpolated video written out into: videos_for_poster/eruption_test_34_interpolated.mp4
Reduced video written out into: videos_for_poster/eruption_test_34_reduced.mp4
Generated vs original video distance is
16005.213431879001
Interpolated vs original video distance is
1819.7101417533509
Original video is: videos/erupting_val/getty-the-voskok-1-becomes-the-first-manned-spaceflight-video-id450027634_46.mp4
Generated video is: generated_TEST_videos/generated_CVAE_videos/CVAE_generated_0035.mp4
Interpolated video written out into: videos_for_poster/eruption_test_35_interpolated.mp4
Reduced video written out into: videos_for_poster/eruption_test_35_reduced.mp4
Generated vs original video distance is
10172.499692799209
Interpolated vs original video distance is
8136.9485681058595
Original video is: videos/erupting_val/flickr-1-3-3-7-8-6-3-3-18513378633_4.mp4
Generated video is: generated_TEST_videos/generated_CVAE_videos/CVAE_generated_0036.mp4
Interpolated video written out into: videos_fo

Interpolated video written out into: videos_for_poster/eruption_test_51_interpolated.mp4
Reduced video written out into: videos_for_poster/eruption_test_51_reduced.mp4
Generated vs original video distance is
12909.864367993956
Interpolated vs original video distance is
5126.729756872309
Original video is: videos/erupting_val/getty-hot-spring-video-id185675476_5.mp4
Generated video is: generated_TEST_videos/generated_CVAE_videos/CVAE_generated_0052.mp4
Interpolated video written out into: videos_for_poster/eruption_test_52_interpolated.mp4
Reduced video written out into: videos_for_poster/eruption_test_52_reduced.mp4
Generated vs original video distance is
16484.569421128355
Interpolated vs original video distance is
3784.1836636188787
Original video is: videos/erupting_val/getty-group-of-eoraptors-grazing-in-field-video-id1319-11_9.mp4
Generated video is: generated_TEST_videos/generated_CVAE_videos/CVAE_generated_0053.mp4
Interpolated video written out into: videos_for_poster/eruption_

Interpolated video written out into: videos_for_poster/eruption_test_68_interpolated.mp4
Reduced video written out into: videos_for_poster/eruption_test_68_reduced.mp4
Generated vs original video distance is
9634.967877476292
Interpolated vs original video distance is
1192.8952175275078
Original video is: videos/erupting_val/getty-mount-aso-video-id483913210_8.mp4
Generated video is: generated_TEST_videos/generated_CVAE_videos/CVAE_generated_0069.mp4
Interpolated video written out into: videos_for_poster/eruption_test_69_interpolated.mp4
Reduced video written out into: videos_for_poster/eruption_test_69_reduced.mp4
Generated vs original video distance is
12710.341734194246
Interpolated vs original video distance is
3436.012077976444
Original video is: videos/erupting_val/getty-geyser-in-iceland-video-id523120502_9.mp4
Generated video is: generated_TEST_videos/generated_CVAE_videos/CVAE_generated_0070.mp4
Interpolated video written out into: videos_for_poster/eruption_test_70_interpolat

Interpolated video written out into: videos_for_poster/eruption_test_86_interpolated.mp4
Reduced video written out into: videos_for_poster/eruption_test_86_reduced.mp4
Generated vs original video distance is
7904.904933014691
Interpolated vs original video distance is
2622.591657120872
Original video is: videos/erupting_val/yt-SUNwIm3o-64_19.mp4
Generated video is: generated_TEST_videos/generated_CVAE_videos/CVAE_generated_0087.mp4
Interpolated video written out into: videos_for_poster/eruption_test_87_interpolated.mp4
Reduced video written out into: videos_for_poster/eruption_test_87_reduced.mp4
Generated vs original video distance is
25441.85030613929
Interpolated vs original video distance is
15562.786639930524
Original video is: videos/erupting_val/getty-cotopaxi-volcano-ecuador-erupting-water-vapour-in-november-2015-video-id506205424_4.mp4
Generated video is: generated_TEST_videos/generated_CVAE_videos/CVAE_generated_0088.mp4
Interpolated video written out into: videos_for_poster/

In [12]:
print("avg CVAE generated L2 dist = {} ".format(np.mean(L2generated)))
print("SD of CVAE generated L2 dist = {} ".format(np.std(L2generated)))

avg CVAE generated L2 dist = 18306.95313934595 
SD of CVAE generated L2 dist = 10894.311713173593 


In [11]:
print("avg interpolated L2 dist = {} ".format(np.mean(L2interpolated)))
print("SD of interpolated L2 dist = {} ".format(np.std(L2interpolated)))

avg interpolated L2 dist = 6248.456892824278 
SD of interpolated L2 dist = 5764.934314328028 


avg CVAE generated L2 dist = 10894.311713173593 


In [16]:
"GAN generated"

gan_titles_raw = ['videos/erupting_val/getty-aerial-track-over-erupting-old-faithful-geyser-in-winter-yellowstone-video-id595896631_2.mp4', 'videos/erupting_val/yt-SUNwIm3o-64_19.mp4', 'videos/erupting_val/getty--video-id104511618_18.mp4', 'videos/erupting_val/flickr-1-2-7-2-1-9-4-2-2912721942_9.mp4', 'videos/erupting_val/getty-close-up-fireworks-video-id628285946_1.mp4', 'videos/erupting_val/giphy-xT4uQbhogM1sS5njXi_0.mp4', 'videos/erupting_val/getty-close-up-of-steaming-lava-dome-at-merapi-volcano-central-java-29-video-id150962577_1.mp4', 'videos/erupting_val/getty-close-up-vigorously-erupting-lava-lake-in-marum-volcano-marum-volcano-video-id150962413_2.mp4', 'videos/erupting_val/getty-large-eruption-at-sakurajima-volcano-in-japan-sends-volcanic-ash-video-id464370844_11.mp4', 'videos/erupting_val/yt-Y_9vd4HWlVA_404.mp4', 'videos/erupting_val/getty-geyser-hole-filled-of-boiling-water-before-eruption-iceland-video-id456462830_6.mp4', 'videos/erupting_val/getty-volcanic-eruptions-on-mt-etna-sicily-italy-video-id157397360_10.mp4', 'videos/erupting_val/getty-fighting-subsided-in-war-torn-gaza-monday-at-the-start-of-the-muslim-video-id453004362_19.mp4', 'videos/erupting_val/flickr-4-0-3-7-4-0-5-7-15940374057_128.mp4', 'videos/erupting_val/giphy-V05ReDqkS1VCg_4.mp4', 'videos/erupting_val/getty-geysers-at-yellowstone-national-park-wyoming-usa-video-id83264515_16.mp4', 'videos/erupting_val/giphy-sUeU52JKc8one_2.mp4', 'videos/erupting_val/flickr-5-5-2-2-8-9-2-8-14255228928_19.mp4', 'videos/erupting_val/flickr-7-7-9-1-3-6-9-6-19077913696_14.mp4', 'videos/erupting_val/getty-cotopaxi-volcano-ecuador-erupting-water-vapour-in-november-2015-video-id506205424_4.mp4', 'videos/erupting_val/getty-bardarbunga-volcano-eruption-and-lava-flow-video-id626439064_19.mp4', 'videos/erupting_val/getty-geysers-atacama-desert-video-id636281892_3.mp4', 'videos/erupting_val/flickr-4-6-7-8905106467_44.mp4', 'videos/erupting_val/getty-medium-shot-girl-posing-with-blue-ribbon-next-to-model-of-erupting-video-id873-100_8.mp4', 'videos/erupting_val/getty-grey-smoke-and-ash-cloud-billow-from-crater-into-sky-zoom-in-to-mount-video-id143541944_89.mp4', 'videos/erupting_val/getty-aerials-active-volcano-shinmoedake-video-id459964798_41.mp4', 'videos/erupting_val/flickr-1-1-3-2-1-3-2-5-15911321325_26.mp4', 'videos/erupting_val/getty-krakatau-video-id493322240_4.mp4', 'videos/erupting_val/getty-aerial-view-of-molten-lava-flow-on-the-island-of-hawaii-video-id640164583_1.mp4', 'videos/erupting_val/getty-sakurajima-volcano-erupts-violently-with-a-large-explosive-eruption-video-id464368814_13.mp4', 'videos/erupting_val/getty-large-and-dangerous-pyroclastic-flow-sweeps-down-the-side-of-sinabung-video-id478438408_4.mp4', 'videos/erupting_val/giphy-Hssp6w4SfuOB2_10.mp4', 'videos/erupting_val/getty-geyser-in-iceland-video-id523120502_9.mp4', 'videos/erupting_val/getty-lava-erupting-from-earth-in-smoking-lava-field-hawaii-video-id1B02924_0004_7.mp4', 'videos/erupting_val/getty-flames-rise-from-a-volcanic-eruption-on-fernandina-island-available-video-id3B2FBBA3_0001_5.mp4', 'videos/erupting_val/getty-cliff-geyser-erupting-video-id685738891_4.mp4', 'videos/erupting_val/getty-hot-spring-video-id185675476_5.mp4', 'videos/erupting_val/getty-mtbromo-time-lapse-video-id613371200_4.mp4', 'videos/erupting_val/5-1-1-5-7-8-7-1-6151157871_5.mp4', 'videos/erupting_val/getty-smoke-and-flames-rise-from-ship-after-explosion-at-gadani-yard-on-1-video-id620102418_83.mp4', 'videos/erupting_val/getty-film-can-the-birth-of-a-nation-clip-battle-field-lillian-gish-sitting-video-id509496583_9.mp4', 'videos/erupting_val/getty-man-sits-on-ledge-overlooking-erupting-lava-lake-marum-volcano-ambrym-video-id150962447_2.mp4', 'videos/erupting_val/getty-major-volcanic-eruption-and-ash-cloud-at-mount-sinabung-video-id498515125_7.mp4', 'videos/erupting_val/meta-1395624_51.mp4', 'videos/erupting_val/flickr-6-5-8-1-9-6-9-9-16365819699_53.mp4', 'videos/erupting_val/getty-eruption-of-the-shinmoedake-crater-of-the-kirishima-volcano-japan-28-video-id150975548_14.mp4', 'videos/erupting_val/getty-jaguar-crashes-through-a-concrete-barrier-video-id698871039_14.mp4', 'videos/erupting_val/getty-volcanic-ash-erupts-vigorously-from-the-crater-at-sakurajima-volcano-video-id695848030_12.mp4', 'videos/erupting_val/flickr-5-3-9-9-6-7-3-0-5553996730_22.mp4', 'videos/erupting_val/getty-close-up-shot-of-steam-pouring-from-crater-of-sakurajima-volcano-at-video-id150962351_11.mp4', 'videos/erupting_val/flickr-7-0-0-3-7-1-1-8-21470037118_126.mp4', 'videos/erupting_val/flickr-1-3-3-7-8-6-3-3-18513378633_4.mp4', 'videos/erupting_val/flickr-0-5-7-3-1-3-2-1-21105731321_9.mp4', 'videos/erupting_val/giphy-qufwQbBoDhNYc_12.mp4', 'videos/erupting_val/getty-el-volcan-calbuco-registro-dos-violentas-y-sorpresivas-erupciones-en-video-id471199520_34.mp4', 'videos/erupting_val/getty-bubbling-flowing-redhot-lava-after-the-eruption-of-kilauea-volcano-video-id85200270_11.mp4', 'videos/erupting_val/flickr-9-7-6-6-1-2-1-7-9497661217_62.mp4', 'videos/erupting_val/getty-timelapsed-of-sunrise-at-mount-bromo-volcano-the-magnificent-view-of-video-id545326216_3.mp4', 'videos/erupting_val/giphy-1cdrsxfhNHfAQ_2.mp4', 'videos/erupting_val/getty-yasur-volcano-video-id465612842_2.mp4', 'videos/erupting_val/getty-sea-hell-in-beppu-japan-video-id473184667_1.mp4', 'videos/erupting_val/getty-eruption-on-mt-etna-13-october-98-sicily-italy-video-id157397349_2.mp4', 'videos/erupting_val/getty-aerials-active-volcano-aso-video-id459967068_1.mp4', 'videos/erupting_val/getty-close-up-of-firey-explosion-video-id307-291_1.mp4', 'videos/erupting_val/getty-lava-surges-in-crater-at-dusk-marum-volcano-ambrym-island-vanuatu-video-id150962395_8.mp4', 'videos/erupting_val/getty-hot-water-spurting-from-geyser-yellowstone-national-park-wyoming-video-id141030434_2.mp4', 'videos/erupting_val/getty-lava-flows-through-a-channel-available-in-hd-video-id3B2FBBA7_0002_4.mp4', 'videos/erupting_val/getty-eruzione-di-una-bocca-del-vulcano-stromboli-video-id847691400_13.mp4', 'videos/erupting_val/getty-big-ash-column-rises-from-shinmoedake-volcano-at-night-mount-in-of-video-id478689742_6.mp4', 'videos/erupting_val/getty-algal-mat-grows-at-edge-of-hot-geyser-pool-yellowstone-usa-video-id564828225_9.mp4', 'videos/erupting_val/giphy-ClG9uYb0hWzPq_0.mp4', 'videos/erupting_val/getty-mount-bromo-volcano-in-east-java-indonesia-video-id649447834_3.mp4', 'videos/erupting_val/getty-eruption-of-volcano-bromo-in-indonesia-video-id461618460_4.mp4', 'videos/erupting_val/yt-Hj9wcztisDU_787.mp4', 'videos/erupting_val/getty-from-encyclopaedia-britannica-films-documentary-depicting-the-study-video-id85200310_4.mp4', 'videos/erupting_val/getty-the-voskok-1-becomes-the-first-manned-spaceflight-video-id450027634_46.mp4', 'videos/erupting_val/getty-eruption-on-krakatau-video-id493455112_3.mp4', 'videos/erupting_val/flickr-0-6-6-5-1-0-5-3-5906651053_10.mp4', 'videos/erupting_val/getty-clouds-scud-over-clepsydra-geyser-erupting-in-yellowstone-national-video-id3B2FBBED_0155_3.mp4', 'videos/erupting_val/getty-camera-pans-right-to-geyser-erupting-spraying-white-water-and-grey-video-id1B02172_0004_13.mp4', 'videos/erupting_val/getty-view-of-heavy-explosion-in-tropical-island-video-id105186610_10.mp4', 'videos/erupting_val/getty-boiling-water-ejects-from-erupting-castle-geyser-yellowstone-usa-video-id595897161_15.mp4', 'videos/erupting_val/getty-clouds-pass-over-the-summit-of-the-erupting-oldoinyo-lengai-volcano-video-id3B2FBBC6_0597_82.mp4', 'videos/erupting_val/getty-lava-from-a-volcanic-eruption-spews-into-the-night-sky-available-in-video-id3B2FBBA6_0005_67.mp4', 'videos/erupting_val/getty-exterior-night-shots-of-baltimore-police-advancing-with-riot-shields-video-id471929516_45.mp4', 'videos/erupting_val/getty-close-up-large-waves-of-lava-erupting-from-volcano-marum-volcano-video-id150962405_3.mp4', 'videos/erupting_val/getty-aerial-view-of-hot-lava-in-hawaii-volcanoes-national-park-video-id640164559_1.mp4', 'videos/erupting_val/getty-the-24story-monaco-tower-at-the-shuttered-riviera-hotel-casino-is-on-video-id591578470_2.mp4', 'videos/erupting_val/getty-firework-display-in-tokyo-video-id629717708_16.mp4', 'videos/erupting_val/getty-mount-aso-video-id483913210_8.mp4', 'videos/erupting_val/getty-ash-explodes-from-crater-of-stromboli-volcano-close-up-shot-stromboli-video-id150962340_13.mp4', 'videos/erupting_val/getty-boardwalk-in-volcanic-wonderland-video-id472597513_7.mp4', 'videos/erupting_val/getty-bright-moon-shining-behind-eruption-plume-of-geyser-video-id685739069_19.mp4', 'videos/erupting_val/getty-aerial-track-towards-and-over-smoking-volcano-and-crater-video-id1B03393_0004_1.mp4', 'videos/erupting_val/getty-bright-explosion-with-fairydust-aftereffect-video-iddv389020_1.mp4', 'videos/erupting_val/getty-november-volcano-erupts-in-columbia-lib-columbia-extnight-nevada-del-video-id675358808_7.mp4', 'videos/erupting_val/getty-group-of-eoraptors-grazing-in-field-video-id1319-11_9.mp4', 'videos/erupting_val/an-erupting-volcano-sends-lava-spewing-into-the-sky-available-in-hd-video-id3B2FBBA6_0010.mp4', 'videos/erupting_val/getty-geyser-erupts-and-splashes-video-id685738819_1.mp4', 'videos/erupting_val/getty-grey-smoke-and-ash-cloud-rising-quickly-upwards-from-crater-zoom-out-video-id143541934_2.mp4']

In [26]:
gan_titles = []
for i in range(len(gan_titles_raw)):
    title = gan_titles_raw[i].replace('videos/erupting_val/','')
    gan_titles.append(title)
    
print(gan_titles[1])
print(titlelist[1])


len(list(set(gan_titles).intersection(titlelist)))

yt-SUNwIm3o-64_19.mp4
giphy-qufwQbBoDhNYc_12.mp4


100

In [30]:
"generated_TEST_videos/GAN_erupting_100_600_generated_ordered/GAN_erupting_100_600"

gan_dict = {}
for i in range(len(gan_titles)):
    gan_dict[gan_titles[i]] = i
    


In [35]:
#MAP GAN VIDEOS TO CORRECT CVAE, interpolated, original VIDEOS
for i in range(len(titlelist)):
    # find ith title
    j = gan_dict[titlelist[i]]
    #print(j)
    #print(titlelist[i])
    # jth gan video with that title should output ith gan video
    copyfile("generated_TEST_videos/GAN_erupting_100_600_generated_ordered/GAN_erupting_100_600/GAN_generated_00{:02d}.mp4".format(j), "videos_for_poster_eruption/eruption_test_{:02d}_GAN.mp4".format(i))
    

33
getty-lava-erupting-from-earth-in-smoking-lava-field-hawaii-video-id1B02924_0004_7.mp4
53
giphy-qufwQbBoDhNYc_12.mp4
4
getty-close-up-fireworks-video-id628285946_1.mp4
48
flickr-5-3-9-9-6-7-3-0-5553996730_22.mp4
80
getty-view-of-heavy-explosion-in-tropical-island-video-id105186610_10.mp4
82
getty-clouds-pass-over-the-summit-of-the-erupting-oldoinyo-lengai-volcano-video-id3B2FBBC6_0597_82.mp4
43
meta-1395624_51.mp4
63
getty-close-up-of-firey-explosion-video-id307-291_1.mp4
21
getty-geysers-atacama-desert-video-id636281892_3.mp4
18
flickr-7-7-9-1-3-6-9-6-19077913696_14.mp4
41
getty-man-sits-on-ledge-overlooking-erupting-lava-lake-marum-volcano-ambrym-video-id150962447_2.mp4
46
getty-jaguar-crashes-through-a-concrete-barrier-video-id698871039_14.mp4
30
getty-large-and-dangerous-pyroclastic-flow-sweeps-down-the-side-of-sinabung-video-id478438408_4.mp4
20
getty-bardarbunga-volcano-eruption-and-lava-flow-video-id626439064_19.mp4
73
yt-Hj9wcztisDU_787.mp4
98
getty-geyser-erupts-and-splashe

In [43]:
#os.listdir("videos_for_poster/videos_for_poster_eruption")

['eruption_test_49_CVAE.mp4',
 'eruption_test_48_CVAE.mp4',
 'eruption_test_53_interpolated.mp4',
 'eruption_test_53_reduced.mp4',
 'eruption_test_23_reduced.mp4',
 'eruption_test_80_reduced.mp4',
 'eruption_test_00_CVAE.mp4',
 'eruption_test_01_CVAE.mp4',
 'eruption_test_92_CVAE.mp4',
 'eruption_test_36_interpolated.mp4',
 'eruption_test_93_CVAE.mp4',
 'eruption_test_08_interpolated.mp4',
 'eruption_test_33_GAN.mp4',
 'eruption_test_23_GAN.mp4',
 'eruption_test_77_CVAE.mp4',
 'eruption_test_76_CVAE.mp4',
 'eruption_test_29_interpolated.mp4',
 'eruption_test_17_interpolated.mp4',
 'eruption_test_11_reduced.mp4',
 'eruption_test_61_reduced.mp4',
 'eruption_test_51_GAN.mp4',
 'eruption_test_41_GAN.mp4',
 'eruption_test_94_interpolated.mp4',
 'eruption_test_72_interpolated.mp4',
 'eruption_test_77_GAN.mp4',
 'eruption_test_98_CVAE.mp4',
 'eruption_test_67_GAN.mp4',
 'eruption_test_99_CVAE.mp4',
 'eruption_test_03_interpolated.mp4',
 'eruption_test_66_interpolated.mp4',
 'eruption_test_58_

In [45]:
##CONVERT VIDEOS INTO GIFS

from moviepy.editor import *

for file in os.listdir("videos_for_poster/videos_for_poster_eruption"):
    clip = VideoFileClip("videos_for_poster/videos_for_poster_eruption/"+file)
    clip.write_gif("videos_for_poster/erupting_gifs/"+file.replace("mp4","gif"))

t:  50%|█████     | 15/30 [00:00<00:00, 143.02it/s, now=None]

MoviePy - Building file videos_for_poster/erupting_gifs/eruption_test_49_CVAE.gif with imageio.


t:  37%|███▋      | 11/30 [00:00<00:00, 102.20it/s, now=None]

MoviePy - Building file videos_for_poster/erupting_gifs/eruption_test_48_CVAE.gif with imageio.


t:  40%|████      | 12/30 [00:00<00:00, 119.08it/s, now=None]

...

MoviePy - Building file videos_for_poster/erupting_gifs/eruption_test_87_interpolated.gif with imageio.


t:  93%|█████████▎| 28/30 [00:00<00:00, 139.75it/s, now=None]

MoviePy - Building file videos_for_poster/erupting_gifs/eruption_test_38_reduced.gif with imageio.


t:  43%|████▎     | 13/30 [00:00<00:00, 120.76it/s, now=None]

MoviePy - Building file videos_for_poster/erupting_gifs/eruption_test_48_reduced.gif with imageio.


In [46]:
for file in os.listdir("videos_for_poster/videos_for_poster_skiing"):
    clip = VideoFileClip("videos_for_poster/videos_for_poster_skiing/"+file)
    clip.write_gif("videos_for_poster/skiing_gifs/"+file.replace("mp4","gif"))

t:  43%|████▎     | 13/30 [00:00<00:00, 129.44it/s, now=None]

MoviePy - Building file videos_for_poster/skiing_gifs/skiing_test_66_reduced.gif with imageio.


t:  43%|████▎     | 13/30 [00:00<00:00, 121.01it/s, now=None]

MoviePy - Building file videos_for_poster/skiing_gifs/skiing_test_51_interpolated.gif with imageio.


t:  47%|████▋     | 14/30 [00:00<00:00, 133.14it/s, now=None]

...

MoviePy - Building file videos_for_poster/skiing_gifs/skiing_test_39_CVAE.gif with imageio.


t:  37%|███▋      | 11/30 [00:00<00:00, 100.57it/s, now=None]

MoviePy - Building file videos_for_poster/skiing_gifs/skiing_test_38_CVAE.gif with imageio.


t:  50%|█████     | 15/30 [00:00<00:00, 142.73it/s, now=None]

MoviePy - Building file videos_for_poster/skiing_gifs/skiing_test_80_reduced.gif with imageio.
